# Feature Extraction

In [29]:
import pandas as pd
import numpy as np
import scipy

import librosa
from librosa import display, beat
from librosa.feature import chroma_stft, melspectrogram

import glob 

from beat_features import beat_feat_parser, static_tempo # Importing from .py file to keep things more readable

Testing the function

In [3]:
dnb = r"D:\Music\DnB\beatjunkierato+dnbpanacea1hitfreedl.mp3"
beat = beat_feat_parser(dnb)

### Beat features

The beat features is based on the ["Advanced Usage"](https://librosa.github.io/librosa/tutorial.html#advanced-usage) section in their tutorial.  This extracts features by "integrating harmonic-percussive separation, multiple spectral features, and beat-synchronous feature aggregation." I'll give a (mostly) high level overview of what these mean and why I chose to utilize them.  I'll also explain the behind-the-scenes action of the function because the actual code isn't that easy to understand without some prior knowledge.

In general, using raw features from an audio time series is not the best route to go.  There are many reasons for this (different scales, units, etc) but one example is a problem I encountered before I got this working.  The resultant output of the function gave a different NumPy array for each file.  One axis was constant: it was the index for the feature "names."  The pitch chroma thats computed has 12 components that correspond to the instensity of each pitch (by [pitch](https://en.wikipedia.org/wiki/Pitch_(music)) I mean the familiar C, D, E, F, G, A, B scale).  The other axis was the value of the coefficient at a frame, which is based on the length of the audio file.  Besides the fact that dealing with a 12x16000 tensor isn't particularly fun, the size difference also is an issue for comparing audio tracks.  

This can be handled by aggregating the features in a multitude of ways.  You can do this statistically with the mean, median, standard deviation, etc, or change how you extract the features, or some other clever way that gets you want you need.  Additionally, these aggregations actually can give you more info about the audio file because there could be meaning in the statistical features.  I chose to use the arithmetic mean because it was easy to implement and it would still give a meaningful representation of each song.

Now to look at what was actually aggregated and how I got them.

#### Harmonic-Percussive Source Separation (HPSS)

Without going too much into the math behind this, HPSS takes the Fourier transform of short windows (on the order of ms) that slightly overlap which then allows you to separate the harmonic and percussive elements of the song.  This allows easier extraction of harmonic features like pitch and percussive features like rhythm/tempo.  

#### Mel-frequency cepstral coefficients (MFCCs)

See [LibROSA EDA](./notebooks/Notebooks/EDA_LibROSA.ipynb)

#### MFCC Deltas

The smooth first order differences between the MFCC columns.  (Same as in time series analysis).

#### Beat MFCC Delta

The MFCCs and are aggregated by matching them to a time index (beat frames in this case).

#### Chromagram

Same thing as the [pitch chroma]() from the LibROSA EDA, although this one is computed with a slightly different method.  Explaining the differences would involve too much math for this but you can read more if you're curious: 
- [Short-time Fourier Transform](https://en.wikipedia.org/wiki/Short-time_Fourier_transform)  
- [Constant-Q transform](https://en.wikipedia.org/wiki/Constant-Q_transform)

#### Beat Chroma

Similar to the beat MFCC delta but with the chromagram.  This is also aggregated with the median instead of the mean.

---

The output is a single feature vector for the audio file to be used in the neural network!

In [9]:
beat_features = np.empty((92,))
for files in glob.glob("D:\MusicSubset\*\*.mp3"):
    beat_features = np.vstack((beat_features, beat_feat_parser(files)))

In [27]:
np.savetxt('subset_beat.npy', beat_features, delimiter=',')

In [23]:
features = np.loadtxt('subset_beat.npy', delimiter=',')

In [28]:
features.shape

(240, 92)

In [40]:
tempos = np.empty((1,))
for files in glob.glob("D:\MusicSubset\DnB\*.mp3"):
    tempos = np.vstack((tempos,static_tempo(files, bpm_estimate=150)))